In [2]:

from getpass import getpass
from langchain.chat_models.gigachat import GigaChat

In [3]:
#gigachat_api_key = getpass(prompt='YzZmOGRmMzktOGI1Zi00MmZlLWFjYjEtOWQ2OTc0NjRiN2VlOjVlNDk2MWJiLTJmYjAtNGRjNS04ODkwLTRhNzcyOWVlMjNjNw==')
gigachat_api_key = 'YzZmOGRmMzktOGI1Zi00MmZlLWFjYjEtOWQ2OTc0NjRiN2VlOjVlNDk2MWJiLTJmYjAtNGRjNS04ODkwLTRhNzcyOWVlMjNjNw=='

In [4]:

# Инициализируем языковую модель GigaChat
# verify_ssl_certs=False – без использования сертификатов Минцифры
llm = GigaChat(credentials=gigachat_api_key, verify_ssl_certs=False, temperature=0.01)

In [ ]:
from PyPDF2 import PdfReader
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os
from langchain.vectorstores import FAISS

In [ ]:
def get_pdf_text(pdf_docs):
  reader = PdfReader(pdf_file)
  text = ""
  for page in reader.pages:
    text += page.extract_text()
  return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    return chunks

In [ ]:
pdf_file = "fstek239.pdf"
load = get_pdf_text(pdf_file)

In [ ]:
chunk = get_text_chunks(load)

In [ ]:
print(f"Total documents: {len(chunk)}")

In [ ]:

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [ ]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
        model_name="cointegrated/LaBSE-en-ru",
        model_kwargs={"device": "cpu"}
    )

# Создаем FAISS индекс (базу векторов) с полученными эмбеддингами
db = FAISS.from_texts(chunk, hf_embeddings_model)

In [ ]:

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
import re

In [ ]:
# Создаём простой шаблон
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

In [ ]:
# Создаём промпт из шаблона
prompt = PromptTemplate(template=template, input_variables=["question"])

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


In [ ]:
# Задаем ретривер
retriever=db.as_retriever()

# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
    

In [ ]:
message = chain.invoke("В составе ЗОКИИ имеется котельная, состоящая из ПЛК и АРМ. Каким образом необходимо выполнять требования приказа ФСТЭК России № 239, если установка наложенных СрЗИ невозможна?")
print(message)

## Пример ответа модели с уточняющим вопросом после ответа

In [ ]:
# Создаём шаблон
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

In [ ]:
# Создаём промпт из шаблона
prompt = PromptTemplate(template=template, input_variables=["question"])

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [ ]:
# Задаем ретривер
retriever = db.as_retriever()

# Функция для генерации дополнительных вопросов
def generate_subquestion(question_and_answer):
    main_answer = chain.invoke(f"Посмотри на вопрос и на ответ на него, дополни ответ и напиши его мне. {question_and_answer}")
    return main_answer

# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [ ]:
question = "необходимо ли проводить оценку состояния безопасности ОКИИ по новой методике ФСТЭК России, если у Субъекта КИИ отсутствуют ЗОКИИ?"
answer = chain.invoke(question)
main_answer = generate_subquestion(f"{question} {answer}")

print(main_answer)

In [ ]:
# Загружаем и разбиваем документ
loader = PyPDFLoader("/content/drive/MyDrive/GigaChat_doc/Приказ ФСТЭК России от 25 декабря 2017 г. N 239.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_doc = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

In [5]:
# Загружаем и разбиваем документ
loader = PyPDFLoader("/content/drive/MyDrive/GigaChat_doc/Приказ ФСТЭК России от 25 декабря 2017 г. N 239.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_doc = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")

In [20]:
def get_pdf_text(pdf_docs):
  reader = PdfReader(pdf_file)
  text = ""
  for page in reader.pages:
    text += page.extract_text()
  return text

def get_text_chunks(text):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
    chunks = text_splitter.split_text(text)
    return chunks

In [21]:
pdf_file = "fstek239.pdf"
load = get_pdf_text(pdf_file)

In [22]:
chunk = get_text_chunks(load)

In [23]:
print(f"Total documents: {len(chunk)}")

Total documents: 100


In [24]:

from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS

In [25]:
# Если у вас нет видеокарты, укажите 'device': 'cpu'
hf_embeddings_model = HuggingFaceEmbeddings(
        model_name="cointegrated/LaBSE-en-ru",
        model_kwargs={"device": "cpu"}
    )

# Создаем FAISS индекс (базу векторов) с полученными эмбеддингами
db = FAISS.from_texts(chunk, hf_embeddings_model)

c:\users\professional\pycharmprojects\gigachain\venv\lib\site-packages\transformers\tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


In [26]:

from langchain.prompts import PromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain.schema import StrOutputParser
import re

In [27]:
# Создаём простой шаблон
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

In [28]:
# Создаём промпт из шаблона
prompt = PromptTemplate(template=template, input_variables=["question"])

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])


In [29]:
# Задаем ретривер
retriever=db.as_retriever()

# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)
    

In [31]:
message = chain.invoke("В составе ЗОКИИ имеется котельная, состоящая из ПЛК и АРМ. Каким образом необходимо выполнять требования приказа ФСТЭК России № 239, если установка наложенных СрЗИ невозможна?")
print(message)

Если установка наложенных средств защиты информации (СрЗИ) невозможна, то необходимо выполнить следующие требования приказа ФСТЭК России № 239:

1. Обеспечить безопасность значимого объекта в ходе его эксплуатации. Это включает планирование мероприятий по обеспечению безопасности, анализ угроз безопасности информации, а также реализацию других мер, предусмотренных приказом.

2. Организовать разработку рабочей (эксплуатационной) документации на значимый объект. Эта документация должна содержать описание архитектуры подсистемы безопасности, порядок и параметры настройки программных и программно-аппаратных средств, включая средства защиты информации.

3. При проведении приемочных испытаний значимого объекта и его подсистемы безопасности следует руководствоваться программой и методикой приемочных испытаний. Результаты этих испытаний должны быть включены в акт приемки значимого объекта в эксплуатацию.

4. Если значимый объект является государственной информационной системой, то оценка его б

## Пример ответа модели с уточняющим вопросом после ответа

In [33]:
# Создаём шаблон
template = """
Answer the question based only on the following context: {context}

Question: {question}
"""

In [34]:
# Создаём промпт из шаблона
prompt = PromptTemplate(template=template, input_variables=["question"])

# Объявляем функцию, которая будет собирать строку из полученных документов
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])

In [35]:
# Задаем ретривер
retriever = db.as_retriever()

# Функция для генерации дополнительных вопросов
def generate_subquestion(question_and_answer):
    main_answer = chain.invoke(f"Посмотри на вопрос и на ответ на него, дополни ответ и напиши его мне. {question_and_answer}")
    return main_answer

# Создаём цепочку
chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

In [44]:
question = "необходимо ли проводить оценку состояния безопасности ОКИИ по новой методике ФСТЭК России, если у Субъекта КИИ отсутствуют ЗОКИИ?"
answer = chain.invoke(question)
main_answer = generate_subquestion(f"{question} {answer}")

print(main_answer)

Исходя из представленного контекста, можно сделать вывод, что проведение оценки состояния безопасности ОКИИ по новой методике ФСТЭК России не требуется, если у Субъекта КИИ отсутствуют ЗОКИИ.


In [ ]:
# Загружаем и разбиваем документ
loader = PyPDFLoader("/content/drive/MyDrive/GigaChat_doc/Приказ ФСТЭК России от 25 декабря 2017 г. N 239.pdf")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=200,
)
split_doc = text_splitter.split_documents(documents)
print(f"Total documents: {len(documents)}")